In [2]:
import sys
sys.path.append('../../')
from src.utils.connection import postgres_connection
import pandas as pd

# Data Analisys

## Patient Analisys

In [12]:
query = """
select * from healthcare.refined.patients a 
"""
patients = pd.read_sql(query, con=postgres_connection())

patients.dtypes

patient_id                            int64
name                                 object
age                                 float64
gender                               object
registration_date            datetime64[ns]
date_insertion               datetime64[ns]
age_group                            object
months_since_registration             int64
patient_type                         object
dtype: object

### What is the distribution of patients across age groups?


In [13]:
patients.groupby("age_group").size().reset_index(name="patient_distribution")

,age_group,patient_distribution
0,0-18,1
1,19-30,8
2,31-50,14
3,51-70,12
4,71+,15


### How does the appointment frequency vary by patient type?
A: Regulars are the most frequent patients, who are within the 24-month registration interval.

In [17]:
query = """
select 
	p.patient_type,
	count(p.patient_id) as qtt_patients_by_type
from refined.patients p
left join refined.appointments a on a.patient_id = p.patient_id
group by 1
"""
patients_by_type = pd.read_sql(query, con=postgres_connection())
patients_by_type

,patient_type,qtt_patients_by_type
0,Old,49
1,Regular,57
2,New,6


## Apointment Analisys

### What are the most common appointment types by age group?
A: Checkup is the most common one

In [20]:
query = """
select 
	a.appointment_type,
	p.age_group,
  count(*) as qtt_patients_by_type
from refined.patients p
left join refined.appointments a on a.patient_id = p.patient_id
group by 1, 2 
order by qtt_patients_by_type desc
"""
patients_appointments = pd.read_sql(query, con=postgres_connection())
patients_appointments

,appointment_type,age_group,qtt_patients_by_type
0,Checkup,51-70,14
1,Checkup,71+,13
2,Consultation,31-50,10
3,Emergency,71+,9
4,Emergency,31-50,8
5,Emergency,19-30,8
6,Checkup,31-50,7
7,Emergency,51-70,7
8,Consultation,51-70,6
9,Consultation,19-30,6


### Are there specific days of the week with higher emergency visits?
A: Friday is the one who shows higher emergency visits

In [21]:
query = """
select 
	a.appointment_type,
	a.day_of_week,
	p.age_group,
  count(*) as qtt_patients_by_type
from refined.patients p
left join refined.appointments a on a.patient_id = p.patient_id
where appointment_type = 'Emergency'
group by 1, 2 ,3
order by qtt_patients_by_type desc
"""
patients_appointments = pd.read_sql(query, con=postgres_connection())
patients_appointments

,appointment_type,day_of_week,age_group,qtt_patients_by_type
0,Emergency,Friday,31-50,4
1,Emergency,Friday,19-30,2
2,Emergency,Saturday,71+,2
3,Emergency,Saturday,51-70,2
4,Emergency,Monday,31-50,2
5,Emergency,Friday,51-70,2
6,Emergency,Thursday,19-30,2
7,Emergency,Saturday,19-30,2
8,Emergency,Tuesday,71+,2
9,Emergency,Monday,19-30,2


## Prescription Analysis

### What are the most prescribed medication categories by age group?
A: Heart by the age of 31-50

In [22]:
query = """
select 
	p.medication_category,
	p2.age_group,
	count(*) as qtd_prescription_by_age
from refined.prescriptions p 
left join refined.patients p2 on p2.patient_id = p.patient_id
group by 1, 2
order by qtd_prescription_by_age desc
"""
prescriptions = pd.read_sql(query, con=postgres_connection())
prescriptions

,medication_category,age_group,qtd_prescription_by_age
0,Heart,31-50,16
1,Pain Relief,71+,15
2,Pain Relief,51-70,15
3,Heart,71+,13
4,Heart,51-70,10
5,Diabetes,71+,10
6,Pain Relief,31-50,10
7,Antibiotic,71+,7
8,Pain Relief,19-30,5
9,Antibiotic,51-70,5


### How does prescription frequency correlate with appointment frequency?
A: Yeah, this means that each time a consultation occurs, a prescription is given to the patient.

In [23]:
query = """
with prescription_freq as (
select 
	p.patient_id,
	p.prescription_frequency
from refined.prescriptions p 
left join refined.patients p2 on p2.patient_id = p.patient_id
),
appointment_freq as (
	select 
		patient_id,
		count(*) as appointment_frequency
	from refined.prescriptions p 
	group by 1
		)
select 
	pf1.patient_id,
	prescription_frequency,
	appointment_frequency ,
	prescription_frequency = appointment_frequency as correlation
from prescription_freq pf1
left join appointment_freq pf2 on pf2.patient_id = pf1.patient_id  
"""

prescription_appointment_correlation = pd.read_sql(query, con=postgres_connection())
prescription_appointment_correlation

,patient_id,prescription_frequency,appointment_frequency,correlation
0,15,3,3,True
1,8,7,7,True
2,37,4,4,True
3,23,3,3,True
4,24,5,5,True
...,...,...,...,...
123,8,7,7,True
124,15,3,3,True
125,15,3,3,True
126,22,6,6,True


In [24]:
prescription_appointment_correlation.groupby("correlation").size().reset_index(name="count")

,correlation,count
0,True,128
